In [4]:
# Imports
import pandas as pd
import datetime as dt
import numpy as np

In [ ]:
class PortfolioDB():
    
    # Maintenance margin requirement for each asset (as a fraction of absolute market value)
    maint_margin_frac = 0.5
    
    # Do not modify these
    cash_security_id = 'cash'     # Security identifier for cash
    ret_tallied_dates = set()     # Set of dates we've already tallied
    
    
    # Initialization function (aka constructor) that runs whenever you create a new PortfolioDB() elsewhere, for example: portfolio_db = PortfolioDB()
    def __init__(self) :
        self.positions_df = self.empty_positions_df()
        self.trades_df = self.empty_trades_df()
        self.account_history_df = self.empty_account_history_df()

    def empty_positions_df(self):
        return pd.DataFrame({'security_id': pd.Series([], dtype='object'), #security identifier such as ticker or permno or optionid whatever unique identifier for the position
                             'quantity': pd.Series([], dtype='float'), # number of units in portfolio, can be positive or negative. Never 0 (zero positions removed from this DataFrame unless its cash)
                             'average_cost': pd.Series([], dtype='float'), # average cost of units in portfolio
                             'current_price': pd.Series([], dtype='float') # current price of security
                            })
    
    def empty_trades_df(self):
        return pd.DataFrame({'security_id': pd.Series([], dtype='object'), # ticker or permno or optionid whatever unique identifier for the position
                             'quantity': pd.Series([], dtype='float'), # number of units traded. positive = buy, negative = sell
                             'open_datetime': pd.Series([], dtype='datetime64[ns]'), # datetime on which the trade was opened
                             'open_average_price': pd.Series([], dtype='float'), # average open price of the units in the trade                                              
                             'close_datetime': pd.Series([], dtype='datetime64[ns]'), # datetime on which the trade was closed
                             'close_average_price': pd.Series([], dtype='float') # average close price of the units in the traade
                            })
    
    def empty_account_history_df(self):
        return pd.DataFrame({'datetime': pd.Series([], dtype='datetime64[ns]'), # datetime on which the account data was recorded
                             'nav': pd.Series([], dtype='float'), # account net asset value
                             'cash_position': pd.Series([], dtype='float'), # account cash position
                             'margin_requirement': pd.Series([], dtype='float') # account margin requirement
                            })
        
    def add_cash(self,cash_to_add):        
        if( not self.positions_df['security_id'].str.contains(self.cash_security_id).any() ):
            # Append is being deprecated, so we need to switch to concat
            self.positions_df = pd.concat([ self.positions_df, pd.DataFrame({'security_id':[self.cash_security_id],'quantity':[0], 'average_cost':[1], 'current_price':[1]})])
             # self.positions_df = self.positions_df.append(pd.DataFrame({'security_id':[self.cash_security_id],'quantity':[0], 'average_cost':[1], 'current_price':[1]}))
                

        self.positions_df.loc[self.positions_df['security_id']==self.cash_security_id,'quantity'] += cash_to_add
        
    def open_trades(self,open_trades_df):     
        validate_df_columns(open_trades_df,['security_id','open_datetime','quantity','open_average_price'])
        
        if open_trades_df['security_id'].dtype != object:            
            open_trades_df['security_id'] = open_trades_df['security_id'].astype(object)
                
        open_trades_df = groupby_agg_wavg(df=open_trades_df, group_by=['security_id','open_datetime'],
                                          default_method='first',
                                          wavg_columns=['open_average_price'],
                                          wavg_weight='quantity')

        self.trades_df = pd.concat([ self.trades_df, open_trades_df ])
        
        self.add_cash( -(open_trades_df['open_average_price']*open_trades_df['quantity']).sum() )
            
        trades_position_df = open_trades_df.loc[:,['security_id', 'quantity', 'open_average_price']] 
        trades_position_df = trades_position_df.rename(columns={'open_average_price':'average_cost'}) 
            
        self.append_positions(trades_position_df)
                        
        self.remove_zero_positions() 
        
    def close_trades(self,close_trades_df):
        validate_df_columns(close_trades_df,['security_id','open_datetime','close_datetime','close_average_price'])

        close_trades_df = close_trades_df.groupby(['security_id','open_datetime']).agg('first').reset_index() 
        close_trades_df = close_trades_df.astype({'security_id':'object', 'open_datetime':'datetime64[ns]'})
        
        self.trades_df = self.trades_df.merge(close_trades_df, on=['security_id','open_datetime'], suffixes=('','_cldf'), how='left')

        just_closed = self.trades_df.loc[:,'close_datetime_cldf'].notnull()
    
        self.trades_df.loc[just_closed,'close_datetime'] = pd.to_datetime(self.trades_df.loc[just_closed,'close_datetime_cldf'])
        self.trades_df.loc[just_closed,'close_average_price'] = self.trades_df.loc[just_closed,'close_average_price_cldf']
        
        self.trades_df = self.trades_df.loc[:,~self.trades_df.columns.str.endswith('_cldf')]

        self.add_cash( (self.trades_df.loc[just_closed,'close_average_price']*self.trades_df.loc[just_closed,'quantity']).sum() )
        
        trades_position_df = self.trades_df.loc[just_closed,['security_id', 'quantity', 'open_average_price']]
        trades_position_df = trades_position_df.rename(columns={'open_average_price':'average_cost'}) 
        trades_position_df['quantity'] = trades_position_df['quantity']*(-1) # negate quantity because we are closing
        
        self.append_positions(trades_position_df)     
        
        self.remove_zero_positions() 
    
    def update_prices(self,price_df):
        validate_df_columns(price_df,['security_id','prc','ret'])
        validate_all_values_same(price_df['date'])
                
        if price_df.loc[:,'security_id'].dtype != object:            
            price_df.loc[:,'security_id'] = price_df.loc[:,'security_id'].astype(object)
        
        self.positions_df = self.positions_df.merge(price_df.loc[:,['security_id','prc','ret']],on='security_id',how='left')
        self.positions_df.loc[:,'ret'] = self.positions_df.loc[:,'ret'].fillna(0)
                
        self.positions_df.loc[:,'lagged_price'] = self.positions_df.loc[:,'current_price']
        
        self.positions_df.loc[:,'current_price'] = np.where(self.positions_df.loc[:,'prc'].notnull(),
                                                            self.positions_df.loc[:,'prc'],
                                                            self.positions_df.loc[:,'current_price'])
        
        null_current_price = self.positions_df.loc[:,'current_price'].isna()
        self.positions_df.loc[null_current_price,'current_price'] = self.positions_df.loc[null_current_price,'average_cost']
        
        date = price_df.loc[:,'date'].iloc[0]
        
        if( date not in self.ret_tallied_dates ):
            self.positions_df.loc[:,'ret_based_price'] = self.positions_df.loc[:,'lagged_price']*(self.positions_df.loc[:,'ret']+1)
            
            self.positions_df.loc[:,'quantity_multiple'] = self.positions_df.loc[:,'ret_based_price'] / self.positions_df.loc[:,'current_price']
            self.positions_df.loc[:,'quantity'] = self.positions_df.loc[:,'quantity']*self.positions_df.loc[:,'quantity_multiple']
            
            self.positions_df = self.positions_df.drop(columns=['ret_based_price'])
            
            still_open = self.trades_df.loc[:,'close_datetime'].isna() 
            
            if( not still_open.empty ):  
                self.trades_df = self.trades_df.merge(self.positions_df.loc[:,['security_id','quantity_multiple']],
                                                      on=['security_id'],
                                                      suffixes=('','_posdf'),
                                                      how='left')
                self.trades_df.loc[still_open,'quantity'] = self.trades_df.loc[still_open,'quantity']*self.trades_df.loc[still_open,'quantity_multiple']          
            
                self.trades_df = self.trades_df.loc[:,~self.trades_df.columns.str.endswith('_posdf')]

                self.trades_df = self.trades_df.drop(columns=['quantity_multiple'])                
            
            self.ret_tallied_dates.add(date)
        
        self.positions_df = self.positions_df.drop(columns=['prc','lagged_price','ret'])
    
    def remove_zero_positions(self): 
        prices = np.where( self.positions_df.loc[:,'current_price'].notnull(),
                          self.positions_df.loc[:,'current_price'],
                          self.positions_df.loc[:,'average_cost'] )
        self.positions_df = self.positions_df[ (np.abs(self.positions_df.loc[:,'quantity']*prices) > 1.0E-5) |
                                               (self.positions_df.loc[:,'security_id'] == 'cash') ]
        
    def record_account_data(self,price_df,datetime):
        self.update_prices(price_df)
        self.account_history_df = pd.concat([ self.account_history_df, 
                                             pd.DataFrame({'datetime':[datetime],'nav':[self.current_nav()], 'cash_position':[self.current_cash()], 'margin_requirement':[self.current_margin()]}) ] )
        
    def current_nav(self):
        return (self.positions_df.loc[:,'current_price']*self.positions_df.loc[:,'quantity']).sum()
    
    def current_cash(self):
        return self.positions_df.loc[self.positions_df.loc[:,'security_id']==self.cash_security_id,'quantity'].sum()
    
    def current_margin(self):
        return self.maint_margin_frac*(np.absolute(self.positions_df['current_price']*self.positions_df['quantity']).sum() - abs(self.current_cash()))

    def append_positions(self,new_position_df):
        self.positions_df = pd.concat([ self.positions_df, new_position_df ])
        
        self.positions_df = groupby_agg_wavg(df=self.positions_df, group_by='security_id', default_method='last', wavg_columns=['average_cost'], wavg_weight='quantity')
        
        self.positions_df.loc[ self.positions_df.loc[:,'average_cost'] < 0.1, 'average_cost' ] = np.NaN
        
        
###################################################################
# Helper methods, do not modify
###################################################################

# Groups passed df by the group_by columns and aggregates using:
#   - default_method, unless:
#   - specified_methods, a dictionary mapping column names to methods, contains the column name
#   - the column is contained in wavg_columns, which aggregaes using a custom "weighted average" aggregation
#       - if we are doing weighted average aggregation, wavg_weight is the name of the column used for weighting
def groupby_agg_wavg(df, group_by, default_method='sum', specified_methods={}, wavg_columns=[], wavg_weight=''):    
    starting_col_names = df.columns
    starting_col_types = df.dtypes

    df2 = df.copy() # don't want to modify original passed in
    
    # create dictionary that will map column names to functions used for aggregation
    method_dict = dict.fromkeys(df2.columns,default_method)  # start with using default_method for all columns
    method_dict.update(specified_methods) # update with any specified methods. dict.update modifies the dictionary and does NOT return one
    
    # create columns for weight*var 
    temp_columns = ['wtimes_' + s for s in wavg_columns] # name temp columns wtimes_<original_name>
    df2[temp_columns] = df2[wavg_columns].multiply(df2[wavg_weight], axis='index') # multiply by weights
    method_dict.update( dict.fromkeys(temp_columns,'sum') ) # want to sum these cols when we aggregate
    method_dict[wavg_weight] = 'sum' # need to sum the wavg weight
    
    # run actual groupby, resetting index
    df2 = df2.groupby(group_by).agg(method_dict).reset_index(drop=True)
    
    # use weight*var sum columns to go back to actual weighted average
    df2[wavg_columns] = df2[temp_columns].divide(df2[wavg_weight], axis='index')
    
    # go back to original column names in correct order
    df2 = df2[starting_col_names]
    df2 = df2.astype(starting_col_types)
    
    return df2 

# Validates the passed dataframe has the columns in passed array required_columns
# If not, it raises a ValueError
def validate_df_columns(df,required_columns):
    for col in required_columns:
        if col not in df.columns:
            raise ValueError('DataFrame missing required column named \'' + col + '\'' )

# Validates all the values in the passed series are the same
# If not, it raises a ValueError
def validate_all_values_same(series):
    np_array = series.to_numpy()
    if not (np_array[0] == np_array).all():
        raise ValueError('Series has multiple unique values but is required to have only one')
        
# Validates the passed series has no na entries
# If it has any, it raises a ValueError
def validate_no_missing_values(series):
    if( series.isna().sum() > 0 ):
        raise ValueError('Series has missing value but is required to have none')